<a href="https://colab.research.google.com/github/tangyubin/movie_recommend/blob/master/movie_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import os
import subprocess
import tempfile
import shutil

In [0]:
print(tf.__version__)

In [0]:
sess = tf.InteractiveSession()
a = tf.Variable(0.5)
b = tf.Variable(2.0)
x_1 = tf.placeholder(tf.float32, [None, 1], name = 'x_1')
x_2 = tf.placeholder(tf.float32, [None, 1], name = 'x_2')
y = tf.add(tf.multiply(a, x_1), tf.multiply(b, x_2))


In [0]:
sess.run(tf.global_variables_initializer())
sess.run(y, feed_dict={x_1:[[1]], x_2:[[1]]})

In [0]:
def save_model(sess, ii, oo, version=1):
  MODEL_DIR = tempfile.gettempdir()
  export_path = os.path.join(MODEL_DIR, str(version))
  print('export_path = {}\n'.format(export_path))
  if os.path.isdir(export_path):
    print('\nAlready save a model, cleaning up\n')
    shutil.rmtree(export_path, True)
    print('\nDelete {} completed!\n'.format(export_path))
  tf.saved_model.simple_save(
    sess,
    export_path,
    inputs = ii,
    outputs = oo
  )
  print('\nSave model:\n')
  print(os.listdir(export_path))

In [0]:
save_model(sess, {'x_1':x_1, 'x_2':x_2}, {'y':y})

In [0]:
os.listdir('/tmp/1/variables')

In [0]:
!saved_model_cli show --dir /tmp/1 --all

In [0]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server

In [0]:
os.environ["MODEL_DIR"] = tempfile.gettempdir()

In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [0]:
!tail server.log

In [0]:
import json
req = {
        'signature_name':'serving_default',
        'instances':[
           {
              'x_1':1,
              'x_2':1
           } 
        ]
      }
data = json.dumps(req)

In [0]:
!pip install -q requests

In [0]:
import requests
headers = {"content-type":"application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict',
                             data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

In [0]:
predictions